# 2019年1月の牌譜を使ったトレーニング

In [1]:
import numpy as np
import pandas as pd
from pycaret.classification import *
import os

In [36]:
train_df = pd.read_csv('data/201901.csv')
train_df = train_df.drop('Unnamed: 0', axis=1)
train_df = train_df[train_df.game == 7]
train_df = train_df.drop(['east_end', 'north_end', 'south_end', 'west_end', 'game'], axis=1)

In [8]:
# train_df = pd.read_csv('score.csv', header=None)
# # train_df = train_df[train_df.game == 7]
# # train_df = train_df.drop(['east_end', 'north_end', 'south_end', 'west_end', 'game'], axis=1)
# train_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,250,250,250,250,382,216,268,134,1,0.00,0.00,0.00,2,0
1,245,261,247,247,382,216,268,134,1,-0.16,-0.02,-0.02,2,1
2,235,310,247,208,382,216,268,134,1,-0.75,-0.12,0.27,2,1
3,325,220,247,208,382,216,268,134,1,1.05,0.78,1.17,2,2
4,325,220,272,183,382,216,268,134,1,1.05,0.53,1.42,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206607,200,440,120,240,220,420,120,240,1,-2.40,0.80,-0.40,13,3
206608,250,250,250,250,357,384,105,154,1,0.00,0.00,0.00,7,0
206609,250,340,160,250,357,384,105,154,1,-0.90,0.90,0.00,7,1
206610,265,325,145,255,357,384,105,154,1,-0.60,1.20,0.10,7,2


In [37]:
train_df

,east,north,south,west,game_result
10,264,141,258,337,14
37,301,205,116,378,13
38,296,220,111,373,13
54,472,183,326,19,7
62,393,259,178,170,4
...,...,...,...,...,...
151490,187,61,371,381,23
151502,267,289,328,116,15
151511,191,173,324,312,9
151512,165,251,298,286,9


In [4]:
# df = df[df['round'] == 0]
# df = df.reindex(np.random.permutation(df.index)).reset_index(drop=True)
# # 分割
# ratio = 0.8
# p = int(ratio * len(df)) # 分割点を計算
# train_df = df.iloc[:p, :]
# test_df = df.iloc[p:, :]

In [5]:
train_df.describe()

,east,north,south,west,game_result
count,14009.000000,14009.000000,14009.000000,14009.000000,14009.000000
mean,250.984867,244.826255,251.540867,251.448069,11.480191
std,118.575708,108.166485,120.189769,120.150068,6.925641
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,167.000000,171.000000,166.000000,165.000000,6.000000
50%,244.000000,242.000000,245.000000,245.000000,12.000000
75%,329.000000,312.000000,330.000000,331.000000,17.000000
max,790.000000,721.000000,765.000000,875.000000,23.000000


In [6]:
exp_clf101 = setup(data = train_df, target = 'game_result', session_id=123, silent=True) 

,Description,Value
0,session_id,123
1,Target,game_result
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(14009, 5)"
5,Missing Values,False
6,Numeric Features,4
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [63]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.3754,0.8979,0.3747,0.3773,0.3735,0.3481,0.3483,18.0050
gbc,Gradient Boosting Classifier,0.3611,0.8859,0.3607,0.3636,0.3595,0.3332,0.3334,9.5710
xgboost,Extreme Gradient Boosting,0.3496,0.8849,0.3488,0.3508,0.3482,0.3212,0.3213,5.5670
lightgbm,Light Gradient Boosting Machine,0.3445,0.8848,0.3437,0.3462,0.3430,0.3159,0.3161,1.2220
knn,K Neighbors Classifier,0.3046,0.7635,0.3029,0.3118,0.3023,0.2742,0.2747,0.0430
dt,Decision Tree Classifier,0.2454,0.6062,0.2449,0.2458,0.2445,0.2125,0.2125,0.0290
nb,Naive Bayes,0.2329,0.8244,0.2311,0.2340,0.2125,0.1992,0.2007,0.0190
ada,Ada Boost Classifier,0.2214,0.7849,0.2198,0.2246,0.2088,0.1873,0.1884,0.2510
qda,Quadratic Discriminant Analysis,0.2206,0.8350,0.2201,0.2295,0.2077,0.1867,0.1882,0.5240
lr,Logistic Regression,0.1691,0.7435,0.1644,0.1577,0.1516,0.1321,0.1331,0.4990


In [47]:
model = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.3507,0.8853,0.3503,0.3547,0.3492,0.3223,0.3225
1,0.3425,0.8790,0.3416,0.3474,0.3424,0.3138,0.3140
2,0.3221,0.8779,0.3202,0.3223,0.3200,0.2925,0.2926
3,0.3364,0.8780,0.3366,0.3373,0.3345,0.3075,0.3077
4,0.3527,0.8950,0.3517,0.3534,0.3506,0.3245,0.3247
5,0.3425,0.8851,0.3422,0.3461,0.3418,0.3138,0.3140
6,0.3602,0.8892,0.3606,0.3610,0.3589,0.3323,0.3325
7,0.3531,0.8846,0.3526,0.3591,0.3533,0.3248,0.3250
8,0.3439,0.8883,0.3418,0.3410,0.3415,0.3152,0.3153
9,0.3408,0.8860,0.3391,0.3395,0.3379,0.3120,0.3122


In [71]:
evaluate_model(model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [48]:
res = predict_model(model, data = train_df)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [33]:
pred_df = predict_model(model, data=test_df)
display(pred_df)

NameError: name 'test_df' is not defined

In [15]:
pred_df.describe()

,AB,AC,AD,score,round,Label
count,10634.000000,10634.000000,10634.000000,10634.000000,10634.0,10634.000000
mean,0.353481,0.347309,0.345054,10.403893,0.0,16.705473
std,0.785881,0.758648,0.761334,7.011044,0.0,9.774402
min,-0.810000,-1.000000,-1.300000,0.000000,0.0,0.000000
25%,0.000000,0.000000,0.000000,4.000000,0.0,4.000000
50%,0.000000,0.000000,0.000000,10.000000,0.0,23.000000
75%,0.290000,0.300000,0.290000,16.000000,0.0,23.000000
max,7.260000,7.020000,6.920000,23.000000,0.0,23.000000


In [10]:
def train(game_num):
    train_df = pd.read_csv('data/201901.csv')
    train_df = train_df.drop('Unnamed: 0', axis=1)
    train_df = train_df[train_df.game == 7]
    train_df = train_df.drop(['east_end', 'north_end', 'south_end', 'west_end', 'game'], axis=1)
    
#     学習開始
    exp_clf101 = setup(data = train_df, target = 'game_result', session_id=123, silent=True)
    model = create_model('gbc')
    os.makedirs('models', exist_ok=True)
    save_model(model, 'models/game' + str(game_num))

In [11]:
for game_num in range(8):
    train(game_num)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.3700,0.8866,0.3698,0.3738,0.3688,0.3426,0.3428
1,0.3496,0.8782,0.3496,0.3564,0.3484,0.3213,0.3216
2,0.3089,0.8768,0.3077,0.3115,0.3088,0.2786,0.2787
3,0.3507,0.8795,0.3510,0.3529,0.3489,0.3223,0.3225
4,0.3853,0.8966,0.3862,0.3847,0.3809,0.3585,0.3588
5,0.3619,0.8839,0.3619,0.3652,0.3607,0.3340,0.3343
6,0.3704,0.8878,0.3714,0.3707,0.3683,0.3430,0.3432
7,0.3847,0.8877,0.3839,0.3915,0.3851,0.3578,0.3580
8,0.3704,0.8925,0.3688,0.3710,0.3689,0.3429,0.3430
9,0.3592,0.8890,0.3570,0.3577,0.3566,0.3311,0.3313


Transformation Pipeline and Model Succesfully Saved


# 予測実行オブジェクト

In [4]:
class Ranking:
    def predict(self, game_num):
        model = load_model('models/game' + str(game_num))
        df = pd.DataFrame(np.array([[250,250,250,250]]), columns=['east', 'south', 'west', 'north'])
        display(df)
        res = predict_model(model, data=df)
        display(res)

# テスト実行

In [5]:
ranking = Ranking()
ranking.predict(1)

Transformation Pipeline and Model Successfully Loaded


,east,south,west,north
0,250,250,250,250


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices